In [253]:
import ijson
import itertools
import random
import numpy as np
import sys, os
import pandas as pd 
import torch
from torchsummary import summary
from torchtext import data
import torch.nn as nn
import torch.utils.data
from torch.utils.data import Dataset, TensorDataset,DataLoader
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
# from apex import amp
import shutil

%load_ext autoreload
%autoreload 2
%matplotlib inline


SEED = 7219
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# DATA_DIR = "/Users/suhasgupta/w251/mids-w251-final-project/data/nlp.cs.princeton.edu/SARC/2.0/pol/"
# COMMENTS_FILE=DATA_DIR+"comments.json"
# print(COMMENTS_FILE)


In [17]:
# with open(COMMENTS_FILE, 'r') as f:
#     comments = next(ijson.items(f, ''))

In [18]:
# comments['7uxqr']
# len(comments)

In [19]:
# d = dict(itertools.islice(comments.items(),1000000))
# df = pd.DataFrame.from_dict(d).T
# df.head()
# df.describe()

In [20]:
# df['text'] = df['text'].apply(lambda x: x.lower())
# df['len'] = df['text'].apply(lambda x: len(x.split(" ")))
# import string
# for i in string.punctuation:
#     df['text'] = df['text'].apply(lambda x: x.replace(i, ""))

In [22]:
# df['sign'] = df['score'].apply(lambda x: 1 if x >= 0 else -1)

# # Import the sentiment table for emoticons
# VADER_LEXICON = "/Users/suhasgupta/w251/mids-w251-final-project/models/Tranformer_based/vader_lexicon.txt"
# sen = pd.read_csv(VADER_LEXICON, 
#                    sep='\t',
#                    usecols=[0, 1], 
#                    header=None, 
#                    names=['token', 'polarity'],
#                    index_col='token'
#                   )
# sen.head()


In [23]:
# tidy_format = (
#     df['text']
#     .str.split(expand=True)
#     .stack()
#     .reset_index(level=1)
#     .rename(columns={'level_1': 'num', 0: 'word'})
# )
# tidy_format.head()


In [24]:
# df['polarity'] = (
#     tidy_format
#     .merge(sen, how='left', left_on='word', right_index=True)
#     .reset_index()
#     .loc[:, ['index', 'polarity']]
#     .groupby('index')
#     .sum()
#     .fillna(0)
# )
# df.head()

In [25]:
# df.groupby('sign').describe()

In [26]:
# df.sort_values('score')

In [137]:
# df.to_csv(DATA_DIR+'processed_train_df.csv')

In [17]:
# df_read_from_file = pd.read_csv(data_path+'train_df.csv')
# df_read_from_file = df_read_from_file.rename(columns={"Unnamed: 0": "UniqueId"})
# df_read_from_file.head(10)

,UniqueId,author,created_utc,date,downs,score,subreddit,text,ups,len,sign,polarity
0,7uxqr,Fishbum,1233788424,2009-02,4,0,politics,nancyt pelosi messes up 500 million jobs lost ...,2,19,1,3.6
1,7vewt,jdl2003,1233940024,2009-02,252,1733,politics,netflix ceo please raise my taxes,1985,6,1,1.3
2,7vq9q,[deleted],1234070398,2009-02,23,0,politics,the six million dead jews of world war one,20,9,1,-6.2
3,c07jfvv,Erobern,1234070581,2009-02,0,6,politics,oh right both wars were just jewish conspiraci...,6,14,1,-2.6
4,7w0as,fangolo,1234194112,2009-02,167,891,politics,gop says it is necessary to spend my tax dolla...,1058,26,1,-1.8
5,c07kq5w,_pi,1234237732,2009-02,0,1,politics,do not question the hive mind,1,6,1,0.0
6,c07myx2,jk1150,1234486080,2009-02,0,4,politics,yup all republicans think exactly the same way,4,8,1,0.0
7,7xdys,[deleted],1234625565,2009-02,10,14,politics,wsj begins the jeb bush campaign for 2016,24,8,1,0.0
8,c07o37s,Mastrmind,1234642047,2009-02,0,2,politics,good luck with that,2,4,1,3.9
9,7xvzm,[deleted],1234825236,2009-02,112,129,politics,breaking a crucial campaign promise obama defe...,241,17,1,4.8


In [18]:
# from sklearn.model_selection import train_test_split
# sample_seed = 1234 
# train_df, valid_df = train_test_split(df_read_from_file, test_size=0.2,random_state=sample_seed)

In [20]:
# text_sample = train_df.head(1).text
# print(text_sample)

19664    obama supports new bid to ban assault weapons ...
Name: text, dtype: object


In [23]:
# pd.DataFrame.to_csv(train_df,data_path+'processed_train_df.csv')
# pd.DataFrame.to_csv(valid_df,data_path+'processed_valid_df.csv')

In [14]:
# train_length = len(train_df)
# valid_length = len(valid_df)

In [255]:
# def convert_lines(example,record_len,max_seq_len):
# #     tokenizer = self._tokenizer
# #     max_seq_length = self._MAX_SEQUENCE_LENGTH-2
#     all_tokens = []
#     longer = 0
#     for text in tqdm_notebook(example):
#         tokens_a = tokenizer.tokenize(text)
#         if len(tokens_a)>max_seq_length:
#             tokens_a = tokens_a[:max_seq_length]
#             longer += 1
#         one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
#         all_tokens.append(one_token)
#     print(longer)
#     return np.array(all_tokens)

In [ ]:
# max_seq_len = 128
# train_df_tokenized = convert_lines(train_df["text"].fillna("DUMMY_VALUE"),train_length,max_seq_len)
# train_df_tokenized_tensor = torch.tensor(train_df_tokenized)
# valid_df_tokenized = convert_lines(valid_df["text"].fillna("DUMMY_VALUE"),train_length,max_seq_len)
# valid_df_tokenized_tensor = torch.tensor(valid_df_tokenized)


In [276]:
# torch.save(train_df_tokenized_tensor,DATA_DIR+'/train_df_tokenized_tensor.pt')
# torch.save(valid_df_tokenized_tensor,DATA_DIR+'/valid_df_tokenized_tensor.pt')

In [284]:
# train_data = torch.load(DATA_DIR+'/train_df_tokenized_tensor.pt')
# valid_data = torch.load(DATA_DIR+'/valid_df_tokenized_tensor.pt')